<span id="run-jobs-in-a-batch" />

# Ejecutar trabajos por lotes



Utilice el modo por lotes para enviar varios trabajos primitivos simultáneamente. A continuación se ofrecen ejemplos de trabajo con lotes.

<span id="set-up-to-use-batches" />

## Configurar para usar lotes

Antes de iniciar un lote, debe [configurar Qiskit Runtime](/docs/guides/install-qiskit) e inicializarlo como servicio:



In [1]:
from qiskit_ibm_runtime import (
    QiskitRuntimeService,
    Batch,
    SamplerV2 as Sampler,
    EstimatorV2 as Estimator,
)


service = QiskitRuntimeService()

<span id="open-a-batch" />

## Abrir un lote

Puede abrir un lote en tiempo de ejecución utilizando el gestor de contexto `with Batch(...)` o inicializando la clase `Batch` clase Al iniciar un lote, debe especificar una QPU pasando un objeto `backend` .  El lote se inicia cuando comienza la ejecución de su primer trabajo.

**Clase de lote**



In [2]:
backend = service.least_busy(operational=True, simulator=False)
batch = Batch(backend=backend)
estimator = Estimator(mode=batch)
sampler = Sampler(mode=batch)
# Close the batch because no context manager was used.
batch.close()

**Gestor de contexto**

El gestor contextual abre y cierra automáticamente el lote.



In [3]:
from qiskit_ibm_runtime import (
    Batch,
    SamplerV2 as Sampler,
    EstimatorV2 as Estimator,
)

backend = service.least_busy(operational=True, simulator=False)
with Batch(backend=backend):
    estimator = Estimator()
    sampler = Sampler()

<span id="specify-batch-length" />

<span id="batch-length" />

## Longitud del lote

Puede definir el tiempo máximo de vida (TTL) del lote con el parámetro `max_time` . Esto debería superar el tiempo de ejecución del trabajo más largo. Este temporizador se inicia cuando comienza el lote.  Cuando se alcanza el valor, se cierra el lote.  Los trabajos que se estén ejecutando finalizarán, pero los trabajos que sigan en cola fallarán.

```python
with Batch(backend=backend, max_time="25m"):
  ...
```

También hay un valor de tiempo de vida interactivo (TTL interactivo) que no se puede configurar (1 minuto para todos los planes).  Si no hay trabajos por lotes en cola dentro de esa ventana, el lote se desactiva temporalmente.

Valores TTL máximos por defecto:

| Tipo de instancia        | TTL máximo predeterminado |
| ------------------------ | ------------------------- |
| Todos los planes de pago | 8 horas                   |
| Abrir                    | 10 minutos                |

Para determinar el TTL máximo o el TTL interactivo de un lote, siga las instrucciones de [Determinar detalles del lote](#batch-details) y busque el valor `max_time` o `interactive_timeout` , respectivamente.

<span id="close" />

<span id="close-a-batch" />

## Cerrar un lote

Un lote se cierra automáticamente cuando sale del gestor de contexto. Cuando se sale del gestor contextual de lotes, el lote pasa al estado "En curso, no acepta nuevos trabajos". Esto significa que el lote termina de procesar todos los trabajos en ejecución o en cola hasta que se alcanza el valor TTL máximo. Una vez finalizados todos los trabajos, el lote se cierra inmediatamente. No se pueden enviar trabajos a un lote cerrado.



In [5]:
with Batch(backend=backend) as batch:
    estimator = Estimator()
    sampler = Sampler()
    job1 = estimator.run([estimator_pub])
    job2 = sampler.run([sampler_pub])

# The batch is no longer accepting jobs but the submitted job will run to completion.
result = job1.result()
result2 = job2.result()

<Admonition type="tip">
  Si no utiliza un gestor contextual, cierre manualmente el lote.  Si deja el lote abierto y le envía más trabajos más tarde, es posible que se alcance el TTL máximo antes de que empiecen a ejecutarse los trabajos posteriores, lo que provocará que se cancelen. Puede cerrar un lote en cuanto haya terminado de enviarle trabajos. Cuando un lote se cierra con `batch.close()`, ya no acepta nuevos trabajos, pero los trabajos ya enviados seguirán ejecutándose hasta su finalización y se podrán recuperar sus resultados.
</Admonition>



In [6]:
batch = Batch(backend=backend)

# If using qiskit-ibm-runtime earlier than 0.24.0, change `mode=` to `batch=`
estimator = Estimator(mode=batch)
sampler = Sampler(mode=batch)
job1 = estimator.run([estimator_pub])
job2 = sampler.run([sampler_pub])
print(f"Result1: {job1.result()}")
print(f"Result2: {job2.result()}")

# Manually close the batch. Running and queued jobs will run to completion.
batch.close()

Result1: PrimitiveResult([PubResult(data=DataBin(evs=np.ndarray(<shape=(3, 2), dtype=float64>), stds=np.ndarray(<shape=(3, 2), dtype=float64>), ensemble_standard_error=np.ndarray(<shape=(3, 2), dtype=float64>), shape=(3, 2)), metadata={'shots': 4096, 'target_precision': 0.015625, 'circuit_metadata': {}, 'resilience': {}, 'num_randomizations': 32})], metadata={'dynamical_decoupling': {'enable': False, 'sequence_type': 'XX', 'extra_slack_distribution': 'middle', 'scheduling_method': 'alap'}, 'twirling': {'enable_gates': False, 'enable_measure': True, 'num_randomizations': 'auto', 'shots_per_randomization': 'auto', 'interleave_randomizations': True, 'strategy': 'active-accum'}, 'resilience': {'measure_mitigation': True, 'zne_mitigation': False, 'pec_mitigation': False}, 'version': 2})
Result2: PrimitiveResult([SamplerPubResult(data=DataBin(meas=BitArray(<shape=(3, 2), num_shots=4096, num_bits=2>), meas0=BitArray(<shape=(3, 2), num_shots=4096, num_bits=127>), shape=(3, 2)), metadata={'circ

<span id="batch-details" />

<span id="determine-batch-details" />

## Determinar los detalles del lote

Para obtener una visión general de la configuración y el estado de un lote, incluidos su TTL interactivo y máximo, utilice la página `batch.details() method`.



In [7]:
from qiskit_ibm_runtime import (
    QiskitRuntimeService,
    batch,
    SamplerV2 as Sampler,
)

service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)

with Batch(backend=backend) as batch:
    print(batch.details())

{'id': '1c7e6c9a-9ab3-4eb9-934e-f96f568d92e6', 'backend_name': 'ibm_pinguino2', 'interactive_timeout': 1, 'max_time': 86400, 'active_timeout': 14400, 'state': 'open', 'accepting_jobs': True, 'last_job_started': None, 'last_job_completed': None, 'started_at': None, 'closed_at': None, 'activated_at': None, 'mode': 'batch', 'usage_time': None}


<span id="partition" />

<span id="reconfigure-jobs-for-parallel-processing" />

## Reconfigurar trabajos para procesamiento paralelo

Existen múltiples formas de reconfigurar los trabajos para aprovechar el procesamiento paralelo que proporciona la dosificación. El siguiente ejemplo muestra cómo puede dividir una larga lista de circuitos en varios trabajos y ejecutarlos como un lote para aprovechar el procesamiento en paralelo.



In [31]:
from qiskit_ibm_runtime import SamplerV2 as Sampler, Batch
from qiskit.circuit.random import random_circuit

max_circuits = 100
circuits = [pm.run(random_circuit(5, 5)) for _ in range(5 * max_circuits)]
for circuit in circuits:
    circuit.measure_active()
all_partitioned_circuits = []
for i in range(0, len(circuits), max_circuits):
    all_partitioned_circuits.append(circuits[i : i + max_circuits])
jobs = []
start_idx = 0

with Batch(backend=backend):
    sampler = Sampler()
    for partitioned_circuits in all_partitioned_circuits:
        job = sampler.run(partitioned_circuits)
        jobs.append(job)

<Admonition type="caution">
  Si se establece `backend=backend` en una primitiva, el programa se ejecuta en modo trabajo, incluso si se encuentra dentro de un contexto por lotes o de sesión. La configuración `backend=backend` está obsoleta a partir de Qiskit Runtime v0.24.0.  En su lugar, utilice el `mode` parámetro.
</Admonition>

<span id="next-steps" />

## Pasos siguientes

<Admonition type="tip" title="Recomendaciones">
  *   Pruebe un ejemplo en el tutorial [Combinar opciones de mitigación de errores con la primitiva del estimador](/docs/tutorials/combine-error-mitigation-techniques).
  *   Revise la referencia de [la API de lotes](/docs/api/qiskit-ibm-runtime/batch#batch).
  *   Comprender los [límites del](/docs/guides/job-limits) trabajo al enviar un trabajo a una QPU de IBM.
  *   Revise [las preguntas frecuentes sobre los modos de ejecución.](/docs/guides/execution-modes-faq)
</Admonition>



© IBM Corp., 2017-2026